In [2]:
import numpy as np
import pandas as pd
import pickle
import sys
import os
import collections
import geneTSmunging as gtm
import fdr_control as fc
import network_helpers as nh
import causal_pipeline as cp
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy import misc
import matplotlib.image as mpimg
%matplotlib inline

In [3]:
filenames = ["/Users/jlu96/v-causal-snps/code/della/test-new-union-method-er-norm-ridge-2/test-new-union-method-er-norm-ridge-2-1-network.txt",
            "/Users/jlu96/v-causal-snps/code/della/test-new-union-method-er-norm-ridge-2/test-new-union-method-er-norm-ridge-2-2-network.txt"]

dfs = [pd.read_csv(filename, sep="\t") for filename in filenames]

for df in dfs:
    print df.shape
    print df.head()

(9900, 8)
                            Cause-Effect               Cause  \
0   ENSG00000169174.10-ENSG00000187583.9  ENSG00000169174.10   
1  ENSG00000169174.10-ENSG00000126091.18  ENSG00000169174.10   
2  ENSG00000169174.10-ENSG00000142669.12  ENSG00000169174.10   
3   ENSG00000169174.10-ENSG00000196449.3  ENSG00000169174.10   
4  ENSG00000142669.12-ENSG00000134748.12  ENSG00000142669.12   

               Effect    Weight  AbsWeight   Test  Lag  Coef  
0   ENSG00000187583.9 -0.097885   0.097885  ridge    2     1  
1  ENSG00000126091.18 -0.074229   0.074229  ridge    2     1  
2  ENSG00000142669.12 -0.073594   0.073594  ridge    2     1  
3   ENSG00000196449.3 -0.070261   0.070261  ridge    2     1  
4  ENSG00000134748.12 -0.068475   0.068475  ridge    2     1  
(9900, 8)
                            Cause-Effect               Cause  \
0  ENSG00000169174.10-ENSG00000126091.18  ENSG00000169174.10   
1   ENSG00000116985.9-ENSG00000177868.10   ENSG00000116985.9   
2   ENSG00000183520.10-EN

In [4]:
index_col = "Cause-Effect"
max_col = "AbsWeight"

In [5]:
whole_dfs = []

for df in dfs:
    whole_df = df.set_index([index_col],  drop=False)

    whole_dfs.append(whole_df)

indices = np.unique(np.concatenate(tuple(whole_df.index.values for whole_df in whole_dfs)))

In [8]:
whole_df
print indices.shape

(9900,)


In [11]:
whole_df
print indices.shape

for i in range(len(whole_dfs)):
    whole_df = whole_dfs[i]

    missing_indices = np.setdiff1d(indices, whole_df.index)
    missing_df = pd.DataFrame(data=dict(zip(whole_df.columns.values, [np.NINF for x in whole_df.columns.values])),
                                        index=missing_indices)
    whole_dfs[i] = pd.concat((whole_df, missing_df)).sort_index()
    
    
        # at this point all dfs should have same ordered indices
    print whole_dfs[i].head()

(9900,)
                                       AbsWeight               Cause  \
Cause-Effect                                                           
ENSG00000001460.16-ENSG00000001461.15   0.004096  ENSG00000001460.16   
ENSG00000001460.16-ENSG00000007968.6    0.017500  ENSG00000001460.16   
ENSG00000001460.16-ENSG00000009780.14   0.007030  ENSG00000001460.16   
ENSG00000001460.16-ENSG00000010803.15   0.007109  ENSG00000001460.16   
ENSG00000001460.16-ENSG00000011021.20   0.001773  ENSG00000001460.16   

                                                                Cause-Effect  \
Cause-Effect                                                                   
ENSG00000001460.16-ENSG00000001461.15  ENSG00000001460.16-ENSG00000001461.15   
ENSG00000001460.16-ENSG00000007968.6    ENSG00000001460.16-ENSG00000007968.6   
ENSG00000001460.16-ENSG00000009780.14  ENSG00000001460.16-ENSG00000009780.14   
ENSG00000001460.16-ENSG00000010803.15  ENSG00000001460.16-ENSG00000010803.15   
ENSG000

In [12]:
b = np.array([whole_df[max_col].values for whole_df in whole_dfs])
print b.shape
print b

(2, 9900)
[[ 0.00409632  0.01749961  0.00702979 ...,  0.00893953  0.00237007
   0.00135346]
 [ 0.0006349   0.0204379   0.01517559 ...,  0.01198411  0.01006476
   0.05628179]]


In [13]:
df_maxes = np.argmax(np.array([whole_df[max_col].values for whole_df in whole_dfs]), axis=0)
#print df_maxes
df_indices = [df_maxes == i for i in range(len(whole_dfs))]


max_df = pd.concat(tuple(whole_df[df_index] for df_index, whole_df in zip(df_indices, whole_dfs)))

In [14]:
max_df

,AbsWeight,Cause,Cause-Effect,Coef,Effect,Lag,Test,Weight
Cause-Effect,,,,,,,,
ENSG00000001460.16-ENSG00000001461.15,0.004096,ENSG00000001460.16,ENSG00000001460.16-ENSG00000001461.15,1.0,ENSG00000001461.15,2.0,ridge,-0.004096
ENSG00000001460.16-ENSG00000010803.15,0.007109,ENSG00000001460.16,ENSG00000001460.16-ENSG00000010803.15,1.0,ENSG00000010803.15,2.0,ridge,0.007109
ENSG00000001460.16-ENSG00000053372.4,0.005051,ENSG00000001460.16,ENSG00000001460.16-ENSG00000053372.4,1.0,ENSG00000053372.4,2.0,ridge,-0.005051
ENSG00000001460.16-ENSG00000060656.18,0.031195,ENSG00000001460.16,ENSG00000001460.16-ENSG00000060656.18,1.0,ENSG00000060656.18,2.0,ridge,-0.031195
ENSG00000001460.16-ENSG00000076864.18,0.022249,ENSG00000001460.16,ENSG00000001460.16-ENSG00000076864.18,1.0,ENSG00000076864.18,2.0,ridge,-0.022249
ENSG00000001460.16-ENSG00000085840.11,0.017646,ENSG00000001460.16,ENSG00000001460.16-ENSG00000085840.11,1.0,ENSG00000085840.11,2.0,ridge,-0.017646
ENSG00000001460.16-ENSG00000090020.9,0.010802,ENSG00000001460.16,ENSG00000001460.16-ENSG00000090020.9,1.0,ENSG00000090020.9,2.0,ridge,-0.010802
ENSG00000001460.16-ENSG00000116273.5,0.014111,ENSG00000001460.16,ENSG00000001460.16-ENSG00000116273.5,1.0,ENSG00000116273.5,2.0,ridge,-0.014111
ENSG00000001460.16-ENSG00000116661.9,0.016402,ENSG00000001460.16,ENSG00000001460.16-ENSG00000116661.9,1.0,ENSG00000116661.9,2.0,ridge,-0.016402


In [15]:
max_df[max_df["Cause-Effect"] == "ENSG00000142684.7-ENSG00000160058.17"]

,AbsWeight,Cause,Cause-Effect,Coef,Effect,Lag,Test,Weight
Cause-Effect,,,,,,,,
ENSG00000142684.7-ENSG00000160058.17,0.063657,ENSG00000142684.7,ENSG00000142684.7-ENSG00000160058.17,1.0,ENSG00000160058.17,2.0,ridge,0.063657


In [16]:
max_df[max_df["Cause-Effect"] == "ENSG00000169174.10-ENSG00000196449.3"]

,AbsWeight,Cause,Cause-Effect,Coef,Effect,Lag,Test,Weight
Cause-Effect,,,,,,,,
ENSG00000169174.10-ENSG00000196449.3,0.070261,ENSG00000169174.10,ENSG00000169174.10-ENSG00000196449.3,1.0,ENSG00000196449.3,2.0,ridge,-0.070261
